In [40]:
import random
import numpy as np
from deap import creator, base, tools, algorithms

In [137]:
creator.create('FitnessMax', base.Fitness, weights=(1,))
creator.create('Individual', list, fitness=creator.FitnessMax)

/home/raduviri/anaconda3/envs/chana/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/raduviri/anaconda3/envs/chana/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [138]:
toolbox = base.Toolbox()

In [139]:
toolbox.register('attr_bool', random.randint, 0, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_bool, n=20)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [149]:
def evalOneMax(individual):
#     return (sum(individual),)
    index = len(individual) // 2
    fitness = sum(individual[index:]) - sum(individual[:index])
    return (fitness, )

In [150]:
toolbox.register('evaluate', evalOneMax)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

In [151]:
NGEN = 100
population = toolbox.population(n=300)
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    print(offspring[3])
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = np.array([fit])
    print(offspring[3].fitness.values)
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

[0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
(array([4.]),)
[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]
(array([4.]),)
[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
(array([1.]),)
[0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([5.]),)
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1]
(array([4.]),)
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
(array([7.]),)
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([7.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
(array([8.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
(array([9.]),)
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([10.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([10.]),)
[0, 0, 0, 

In [159]:
population = toolbox.population(n=300)
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, verbose=False)

In [161]:
best_individual = tools.selBest(population, k=1)[0]
best_individual, evalOneMax(best_individual)

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (10,))

In [153]:
top10[0], evalOneMax(top10[0])

(10,)